In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/common_functions"

In [0]:
%run "../includes/configuration"

Find race years for which the data is to be reprocessed

In [0]:
race_results = spark.read.parquet(f"{presentation_folder_path}/race_results") \
    .filter(f"file_date = '{v_file_date}'")

In [0]:
race_year_list = df_column_to_list(race_results, 'race_year')


In [0]:
from pyspark.sql.functions import col

race_results = spark.read.parquet(f"{presentation_folder_path}/race_results") \
    .filter(col('race_year').isin(race_year_list))

In [0]:
from pyspark.sql.functions import sum, desc, when, count, col

constructor_standings = race_results \
    .groupBy('race_year', 'team') \
    .agg(sum('points').alias('Points'), count(when(col('position') == 1, True)).alias('Wins')) \
    .orderBy(desc('Wins'))

In [0]:
display(constructor_standings)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc

constructor_rank_spec = Window.partitionBy('race_year').orderBy(desc('Points'), desc('Wins'))
final_df = constructor_standings \
    .withColumn('rank', rank().over(constructor_rank_spec))
display(final_df)

In [0]:
overwrite_partition(final_df, 'f1_presentation', 'constructor_standings', 'race_year')